## Testing ways to simulate scanning from the Atacama desert

Insert text

In [1]:
import os, pickle, time
import numpy as np
import healpy as hp
from beamconv import ScanStrategy, tools
import matplotlib.pyplot as plt
import qpoint as qp
import seaborn as sns
sns.set()
%matplotlib inline

In [2]:
def get_cls(fname='../ancillary/wmap7_r0p03_lensed_uK_ext.txt'):
    cls = np.loadtxt(fname, unpack=True) # Cl in uK^2
    return cls[0], cls[1:]

In [4]:
# Load up alm                                                                            
lmax = 700
ell, cls = get_cls()
np.random.seed(39)
alm = hp.synalm(cls, lmax=lmax, new=True, verbose=True) # uK  

In [5]:
# create Beam properties and pickle (this is just to test load_focal_plane)              
opj = os.path.join
blm_dir = os.path.abspath('../tests/test_data/example_blms')
po_file = opj(blm_dir, 'blm_hp_X1T1R1C8A_800_800.npy')
eg_file = opj(blm_dir, 'blm_hp_eg_X1T1R1C8A_800_800.npy')

beam_file = 'beam_opts.pkl'
beam_opts = dict(az=0,
                 el=0,
                 polang=0.,
                 btype='Gaussian',
                 name='X1T1R1C8',
                 fwhm=32.2,
                 lmax=800,
                 mmax=800,
                 amplitude=1.,
                 po_file=po_file,
                 eg_file=eg_file,
                 deconv_q=True,  # blm are SH coeff from hp.alm2map                      
                 normalize=True)

with open(beam_file, 'wb') as handle:
    pickle.dump(beam_opts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# init scan strategy and instrument                                                      

nsamp      = 1000
mmax       = 5
fwhm       = 30.
ra0        = -10 
dec0       = -57.5
az_throw   = 50
scan_speed = 2.8
rot_period = 4.5*60*60
nside_spin = 512

spin_opts = dict(max_spin=2,      # Symmetric beams, so 2 is sufficient
                 nside_spin=256)  # nside of scanned maps 

scan_opts = dict(ra0=[ra0],       # Center of scan
                 dec0=[dec0],
                 az_throw=az_throw,     # Scan width
                 scan_speed=scan_speed)

soscan = ScanStrategy(duration=nsamp, # mission duration in sec.                                  
                  sample_rate=10, # 10 Hz sample rate                                    
                  location='atacama') # South pole instrument                              

soscan.load_focal_plane('.', no_pairs=True)
soscan.allocate_maps(nside=256)

chunks = soscan.partition_mission(0.5*soscan.mlen*soscan.fsamp) 

# Set instrument rotation
#b2.set_instr_rot(period=rot_period, angles=[68, 113, 248, 293])

# Set elevation stepping
#b2.set_el_steps(step_period, steps=[-4, -3, -2, -1, 0, 1, 2, 3, 4, 4])

# Set HWP modulation
#b2.set_hwp_mod(mode='continuous', freq=1.)

In [ ]:
# Generate timestreams with Gaussian beams

#soscan.scan_instrument_mpi(alm, verbose=1, ra0=ra0, dec0=dec0, az_throw=az_throw,
#                       nside_spin=nside_spin, max_spin=mmax)

soscan.scan_instrument_mpi(alm, **scan_opts)
scan_opts = dict(ra0=[ra0+10],       # Center of scan
                 dec0=[dec0+10],
                 az_throw=az_throw,     # Scan width
                 scan_speed=scan_speed)
#soscan.allocate_maps(nside=256)
soscan.scan_instrument_mpi(alm, **scan_opts)

maps, cond = soscan.solve_for_map()

In [ ]:
cond[cond == np.inf] = hp.UNSEEN
cart_opts = dict(unit=r'[$\mu K_{\mathrm{CMB}}$]', lonra=[-100, 100], latra=[-70, 80])
hp.cartview(cond, min=2, max=5, **cart_opts)
hp.cartview(maps[0], min=-250, max=250, **cart_opts)
hp.cartview(maps[1], min=-5, max=5, **cart_opts)
hp.cartview(maps[2], min=-5, max=5, **cart_opts)